In [0]:
# COMMAND ----------
%pip install requests pandas

# COMMAND ----------
import requests
import json
from datetime import datetime

# --- CONFIGURATION ---
STORAGE_ACCOUNT_NAME = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
STORAGE_ACCOUNT_KEY = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
CONTAINER_NAME = "raw"

# Params Ville (Le Havre)
LATITUDE = 49.50273
LONGITUDE = 0.23248
START_DATE = "2025-09-19"
END_DATE = "2026-01-15"

# --- SETUP CONNEXION AZURE ---
spark.conf.set(
    f"fs.azure.account.key.{STORAGE_ACCOUNT_NAME}.dfs.core.windows.net",
    STORAGE_ACCOUNT_KEY
)

# =============================================================================
# 1. FONCTION UPLOAD (VERSION DATABRICKS)
# =============================================================================
def upload_to_datalake(data, filename):
    path = f"abfss://{CONTAINER_NAME}@{STORAGE_ACCOUNT_NAME}.dfs.core.windows.net/{filename}"
    try:
        json_str = json.dumps(data)
        dbutils.fs.put(path, json_str, overwrite=True)
        print(f"✅ Météo stockée sur Azure : {path}")
    except Exception as e:
        print(f"❌ Erreur Upload : {e}")
        raise e

# =============================================================================
# 2. LOGIQUE MÉTEO
# =============================================================================
def get_weather_data():
    full_data = []
    
    # A. HISTORIQUE
    print("1️⃣ Récupération Historique...")
    url_hist = "https://archive-api.open-meteo.com/v1/archive"
    params_hist = {
        "latitude": LATITUDE, "longitude": LONGITUDE,
        "start_date": START_DATE, "end_date": END_DATE,
        "hourly": "temperature_2m,relative_humidity_2m,wind_speed_10m,wind_direction_10m,precipitation",
        "timezone": "Europe/Paris"
    }
    
    try:
        resp = requests.get(url_hist, params=params_hist).json()
        h = resp['hourly']
        for i in range(len(h['time'])):
            full_data.append({
                "date_heure": h['time'][i],
                "temperature": h['temperature_2m'][i],
                "humidite": h['relative_humidity_2m'][i],
                "vent_vitesse": h['wind_speed_10m'][i],
                "vent_direction": h['wind_direction_10m'][i],
                "precipitation": h['precipitation'][i],
                "type": "historique"
            })
    except Exception as e:
        print(f"❌ Erreur Historique: {e}")

    # B. PRÉVISIONS (15 JOURS)
    print("2️⃣ Récupération Prévisions (15j)...")
    url_for = "https://api.open-meteo.com/v1/forecast"
    params_for = {
        "latitude": LATITUDE, "longitude": LONGITUDE,
        "hourly": "temperature_2m,relative_humidity_2m,wind_speed_10m,wind_direction_10m,precipitation",
        "forecast_days": 15,
        "timezone": "Europe/Paris"
    }
    
    try:
        resp = requests.get(url_for, params=params_for).json()
        h = resp['hourly']
        for i in range(len(h['time'])):
            full_data.append({
                "date_heure": h['time'][i],
                "temperature": h['temperature_2m'][i],
                "humidite": h['relative_humidity_2m'][i],
                "vent_vitesse": h['wind_speed_10m'][i],
                "vent_direction": h['wind_direction_10m'][i],
                "precipitation": h['precipitation'][i],
                "type": "prevision"
            })
    except Exception as e:
        print(f"❌ Erreur Prévisions: {e}")
        
    return full_data

# =============================================================================
# 3. MAIN
# =============================================================================
data = get_weather_data()
if data:
    upload_to_datalake(data, "weather_full_le_havre.json")
else:
    print("⚠️ Aucune donnée météo récupérée.")

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
1️⃣ Récupération Historique...
2️⃣ Récupération Prévisions (15j)...
Wrote 513797 bytes.
✅ Météo stockée sur Azure : abfss://raw@sadatalakeair2025.dfs.core.windows.net/weather/weather_full_le_havre.json
